# Thinking 1 : XGBoost与GBDT的区别是什么？

答：  
XGBoost 本质上是 GBDT 的工程版本，在 GBDT 的基础上加入了许多模型的优化策略
1. XGB 的损失函数中添加了正则化项，避免过拟合，泛化能力更好。
2. XGB 的损失函数是用泰勒展开式展开的，不仅用到了一阶导，还是用了二阶导，可以保留更多的目标函数信息，并且加快优化速度。
3. GBDT是给新的基模型寻找新的拟合标签（前面加法模型的负梯度），而xgboost是给新的基模型寻找新的目标函数（目标函数关于新的基模型的二阶泰勒展开）。
4. xgboost加入了和叶子权重的L2正则化项，因而有利于模型获得更低的方差。
5. xgboost增加了自动处理缺失值特征的策略。通过把带缺失值样本分别划分到左子树或者右子树，比较两种方案下目标函数的优劣，从而自动对有缺失值的样本进行划分，无需对缺失特征进行填充预处理。
6. XGB 支持并行处理。

# Thinking 2 : 举一个你之前做过的预测例子
>（用的什么模型，解决什么问题，比如我用LR模型，对员工离职进行了预测，效果如何... 请分享到课程微信群中）

答：
Titanic 乘客是否幸存预测  
幸存与非幸存，是一个分类问题（也可以是一个回归问题，取决于使用的模型）  
使用了Auto-ML，随机森林，deep-fm模型  
初步结果 0.729 效果并不好，但随机森林的泛化能力最强，深度模型有时不一定能够得到很好的效果。  
收获：特征决定模型的上限。特征的构造与选取，往往比模型更加重要。

# Thinking 3 : 请你思考，在你的工作中，需要构建哪些特征（比如用户画像，item特征...），这些特征都包括哪些维度（鼓励分享到微信群中，进行交流）

答：  
以前写过的论文：  
碳排放权价格的影响因素主要包括市场、能源、政策三个方面   
能源指标、政策指标、市场指标、环境指标和国外碳市场价格指标五类影响因素进行分析。  
以月度为单位，结合经济学理论分析，充分考虑时间序列以及面板数据的特性，对各个变量进行充分的检验，  
构造了碳排放交易权价格的影响因素模型，并使用自回归系数不同的FGLS方法进行了实证分析，  
得到了政策因素会对碳排放交易权价格产生显著影响的结论，并给出了相关政策建议。  

# Action 1 ：男女声音识别
>数据集：voice.csv, 3168个录制的声音样本（来自男性和女性演讲者），  
采集的频率范围是0hz-280hz，已经对数据进行了预处理    
一共有21个属性值，请判断该声音是男还是女？  
使用Accuracy作为评价标准  

## 一、使用SVM

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score

#数据加载
data = pd.read_csv("voice.csv")
pd.set_option("display.max_columns", 1000)
#print(data)
# print(data.isna().sum())
# print(data.shape)
# print(data.describe())
print(data.info())
print("男性样本个数：{}".format(data[data.label == 'male'].shape[0]))
print("女性样本个数：{}".format(data[data.label == 'female'].shape[0]))

#分离特征值与label
X = data.iloc[:, :-1]
y = data.iloc[: ,-1]
#print(y)

#标签编码
labelencoder = LabelEncoder()
target = labelencoder.fit_transform(y)
print(target)

#数据规范化
ss = StandardScaler()
X = ss.fit_transform(X)
print(X)

#数据切分
x_train, x_test, y_train, y_test = train_test_split(X, target, test_size = 0.2, random_state = 42)

#使用 SVC(分类模型) 进行分析
svc = SVC()
svc.fit(x_train, y_train)
y_pre = svc.predict(x_test)
print("SVM 预测准确率：{}".format(accuracy_score(y_test, y_pre)))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
meanfreq    3168 non-null float64
sd          3168 non-null float64
median      3168 non-null float64
Q25         3168 non-null float64
Q75         3168 non-null float64
IQR         3168 non-null float64
skew        3168 non-null float64
kurt        3168 non-null float64
sp.ent      3168 non-null float64
sfm         3168 non-null float64
mode        3168 non-null float64
centroid    3168 non-null float64
meanfun     3168 non-null float64
minfun      3168 non-null float64
maxfun      3168 non-null float64
meandom     3168 non-null float64
mindom      3168 non-null float64
maxdom      3168 non-null float64
dfrange     3168 non-null float64
modindx     3168 non-null float64
label       3168 non-null object
dtypes: float64(20), object(1)
memory usage: 519.8+ KB
None
男性样本个数：1584
女性样本个数：1584
[1 1 1 ... 0 0 0]
[[-4.04924806  0.4273553  -4.22490077 ... -1.43142165 -1.41913712
  -1.4547722

# 二、使用XGB

In [13]:
import xgboost as xgb

model1 = xgb.XGBClassifier(max_depth = 15,
                          n_estimators = 10000, 
                          colsample_bytree=0.8, 
                          subsample=0.9,
                          eta=0.01,  
                          seed=42)

model1 = model1.fit(x_train, y_train, 
                  eval_metric='auc', 
                  eval_set=[(x_train, y_train), (x_test, y_test)],
                  verbose=True, #早停法，如果auc在10epoch没有进步就stop
                  early_stopping_rounds=10)


y_pre = model1.predict(x_test)
print("准确率为:{}".format(accuracy_score(y_test, y_pre)))

[0]	validation_0-auc:0.99269	validation_1-auc:0.99174
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.99284	validation_1-auc:0.99195
[2]	validation_0-auc:0.99830	validation_1-auc:0.99354
[3]	validation_0-auc:0.99822	validation_1-auc:0.99403
[4]	validation_0-auc:0.99818	validation_1-auc:0.99409
[5]	validation_0-auc:0.99809	validation_1-auc:0.99430
[6]	validation_0-auc:0.99802	validation_1-auc:0.99432
[7]	validation_0-auc:0.99830	validation_1-auc:0.99646
[8]	validation_0-auc:0.99823	validation_1-auc:0.99647
[9]	validation_0-auc:0.99820	validation_1-auc:0.99645
[10]	validation_0-auc:0.99817	validation_1-auc:0.99641
[11]	validation_0-auc:0.99815	validation_1-auc:0.99653
[12]	validation_0-auc:0.99811	validation_1-auc:0.99654
[13]	validation_0-auc:0.99881	validation_1-auc:0.99642
[14]	validation_0-auc:0.99892	validation_1-auc:0.99659
[15]	validation_0-auc:0.9988

C:\Users\10109\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score

#数据加载
data = pd.read_csv("voice.csv")
pd.set_option("display.max_columns", 1000)

#分离特征值与label
X = data.iloc[:, :-1]
y = data.iloc[: ,-1]
#print(y)

#标签编码
labelencoder = LabelEncoder()
target = labelencoder.fit_transform(y)
# print(target)

#数据规范化
ss = StandardScaler()
X = ss.fit_transform(X)
# print(X)

#数据切分
x_train, x_test, y_train, y_test = train_test_split(X, target, test_size = 0.2, random_state = 42)
#设置参数
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
                         'seed':42
        }

#转换数据结构
train_data = xgb.DMatrix(x_train, label = y_train)
test_data = xgb.DMatrix(x_test, label = y_test)
test = xgb.DMatrix(x_test)
model2 = xgb.train(param, train_data, evals=[(train_data, 'train'), (test_data, 'test')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
y_predict = model2.predict(test)

#转化为二分类
predict = pd.DataFrame(y_predict, columns = ["y_pre"])
predict["y_pre"] = predict["y_pre"].map(lambda x:1 if x>=0.5 else 0)
print("准确率为:{}".format(accuracy_score(y_test, predict["y_pre"])))

[03:36:46] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { boosting_type, subsample_freq } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.99195	test-auc:0.98811
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[25]	train-auc:0.99928	test-auc:0.99650
[50]	train-auc:0.99942	test-auc:0.99643
[75]	train-auc:0.99963	test-auc:0.99747
[100]	train-auc:0.99972	test-auc:0.99740
[125]	train-auc:0.99977	test-auc:0.99750
[150]	train-auc:0.99982	test-auc:0.99750
[175]	train-auc:0.99986	test-auc:0.99741
[200]	train-auc:0.99990	test-auc:0.99734
[225]	train-auc:0.99992	test-auc:0.99740
[250]	train-auc:0.99994	test-auc:0.99749
Stopp

# 本章任务

In [14]:
import xlrd
data = xlrd.open_workbook('L6自测文档.xls')
#通过索引顺序获取
table = data.sheet_by_index(0)

""" 工作表中行/列的操作 """
#获取该sheet中的有效行数
nrows = table.nrows  
print(nrows)
row_index, col_index = 0, 0
# 获取某行信息
for row_index in range(2, nrows-5):
    print(table.row(row_index)[:2])
for row_index in range(nrows-4, nrows):
    print(table.row(row_index)[:2], table.row(row_index)[-2])

21
[text:'工具', text:'LR工具使用']
[text:'工具', text:'SVM工具使用']
[text:'原理', text:'FM模型：FM，FFM，DeepFM，NFM，AFM']
[text:'原理', text:'LR优点及缺点']
[text:'原理', text:'SVM优点及缺点']
[text:'原理', text:'FM与LR区别']
[text:'原理', text:'FM与SVM区别']
[text:'原理', text:'三种方式：Bagging, Boosting, Stacking']
[text:'原理', text:'XGBoost原理']
[text:'原理', text:'LightGBM = XGBoost + Histogram + GOSS + EFB']
[text:'原理', text:'CatBoost = Catgorical + Boost']
[text:'原理', text:'自然梯度理解']
[text:'工具', text:'XGBoost，LightGBM, CatBoost工具使用']
[text:'工具', text:'调试工程 Debug能力']
[text:'Thinking1', text:'XGBoost与GBDT的区别是什么？'] text:'简要说明这两者之间的区别（10points）'
[text:'Thinking1', text:'举一个你之前做过的预测例子（用的什么模型，解决什么问题，比如我用LR模型，对员工离职进行了预测，效果如何... 请分享到课程微信群中）'] text:'简要说明之前做过的例子，用的模型，解决的问题，并且在群里分享（10points）'
[text:'Thinking2', text:'请你思考，在你的工作中，需要构建哪些特征（比如用户画像，item特征...），这些特征都包括哪些维度（鼓励分享到微信群中，进行交流）'] text:'能对工作场景，以及构造的特征进行洞察，在班级群中分享（10points）'
[text:'Action1', text:'男女声音识别\n数据集：voice.csv\n3168个录制的声音样本（来自男性和女性演讲者），采集的频率范围是0hz-280hz，已经对数据进行了预处理\n一共有21个属性值，请判断